In [1]:
import findspark
findspark.init()

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.types import StringType
import pyspark.pandas as ps
import glob
import os

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [3]:
# Spark session & context
spark = SparkSession.builder.master("spark://spark:7077") \
        .appName("jupyter-notebook-index2mongo") \
        .config("spark.driver.memory", "1g") \
        .getOrCreate()

spark

In [4]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")  

In [6]:
df = ps.read_parquet('/home/jovyan/work/data/**/*.parquet.zst')

#df = spark.read.option("overwriteSchema", "true").parquet('/home/jovyan/work/data/**/*.parquet.zst')

In [9]:
type(df)

pyspark.pandas.frame.DataFrame

In [10]:
df.tail(2)

,language,country,job_id,productCategoryAlt,productAltEAN,productPriceBase,productBrand,productCategory,productDate,crawlID,currentURL,productLinkDetail,productEAN,productID,productAltId,productImage,productIsAvailable,productIsBio,productIsFresh,productIsFreezer,productIsNew,promotion,isSponsored,productLotPriceBase,productLotPrice,productName,productNutriscore,other,productPackaging,typeCrawler,productPosition,productPricePrevious,productPrice,productPromotionText,productQuantityRating,productRating,enseigneID,enseigneName,productService,shopID,productStock,productUnit,productValueUnit,variant_master,variant_position,zip_code
38472443,FR,FR,379974/15/185782,NaN,NaN,NaN,PTIT BASILE,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",2022-12-29T00:25:51Z,417ca1d7-3878-437d-a6a9-13d82f4ced78,https://www.auchan.fr/bebe/puericulture/chambr...,https://www.auchan.fr/ptit-basile-gigoteuse-et...,None,CA1129202,None,https://media.auchan.fr/4bbea478-5a65-44d3-b75...,1,0,0,0,0,0,NaN,NaN,NaN,Gigoteuse été jersey bio imprimé mini stars,NaN,"{""is_heard_on_radio"": false}",1 coloris,WEB,54,NaN,29.9,None,NaN,None,1,Auchan Drive,NaN,131774,NaN,None,NaN,NaN,NaN,None
38472444,FR,FR,379974/15/185782,NaN,NaN,NaN,PTIT BASILE,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",2022-12-29T00:25:55Z,417ca1d7-3878-437d-a6a9-13d82f4ced78,https://www.auchan.fr/bebe/puericulture/chambr...,https://www.auchan.fr/ptit-basile-lot-x3-draps...,None,CA1306628,None,https://media.auchan.fr/25a25b98-be4e-4974-98c...,1,0,0,0,0,0,NaN,NaN,NaN,Lot x3 draps housse jersey lit bébé 100% coton...,NaN,"{""is_heard_on_radio"": false}",4 coloris,WEB,55,NaN,29.6,None,NaN,None,1,Auchan Drive,NaN,131774,NaN,None,NaN,NaN,NaN,None


In [11]:
df.tail(5)

,language,country,job_id,productCategoryAlt,productAltEAN,productPriceBase,productBrand,productCategory,productDate,crawlID,currentURL,productLinkDetail,productEAN,productID,productAltId,productImage,productIsAvailable,productIsBio,productIsFresh,productIsFreezer,productIsNew,promotion,isSponsored,productLotPriceBase,productLotPrice,productName,productNutriscore,other,productPackaging,typeCrawler,productPosition,productPricePrevious,productPrice,productPromotionText,productQuantityRating,productRating,enseigneID,enseigneName,productService,shopID,productStock,productUnit,productValueUnit,variant_master,variant_position,zip_code
38472440,FR,FR,379974/15/185782,NaN,NaN,NaN,Ptit Albatros,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",2022-12-29T00:25:47Z,417ca1d7-3878-437d-a6a9-13d82f4ced78,https://www.auchan.fr/bebe/puericulture/chambr...,https://www.auchan.fr/ptit-albatros-p-tit-alba...,3454980150250,221894,None,https://media.auchan.fr/433deab3-860e-4e78-85f...,1,0,0,0,0,0,NaN,NaN,NaN,P'tit Albatros - Plan Incliné Bébé 60x35 cm - ...,NaN,"{""is_heard_on_radio"": false}",None,WEB,50,NaN,19.90,None,NaN,None,1,Auchan Drive,NaN,131774,NaN,None,NaN,NaN,NaN,None
38472441,FR,FR,379974/15/185782,NaN,NaN,NaN,PTIT BASILE,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",2022-12-29T00:25:48Z,417ca1d7-3878-437d-a6a9-13d82f4ced78,https://www.auchan.fr/bebe/puericulture/chambr...,https://www.auchan.fr/ptit-basile-drap-bebe-co...,None,CA1306562,None,https://media.auchan.fr/64b2d50e-bb2f-4a5a-ab2...,1,0,0,0,0,0,NaN,NaN,NaN,Drap bébé coton Bio 118x180 cm,NaN,"{""is_heard_on_radio"": false}",17 coloris,WEB,52,NaN,25.90,None,NaN,None,1,Auchan Drive,NaN,131774,NaN,None,NaN,NaN,NaN,None
38472442,FR,FR,379974/15/185782,NaN,NaN,NaN,BLANREVE,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",2022-12-29T00:25:49Z,417ca1d7-3878-437d-a6a9-13d82f4ced78,https://www.auchan.fr/bebe/puericulture/chambr...,https://www.auchan.fr/blanreve-couette-bebe-co...,None,CA985272,None,https://media.auchan.fr/5a29a493-5aac-46f5-8ea...,1,0,0,0,0,1,NaN,NaN,NaN,Couette bébé coton tempérée anti-acariens BAMBIN,NaN,"{""is_heard_on_radio"": false}",1 coloris,WEB,53,34.990002,24.49,-30 %,NaN,None,1,Auchan Drive,NaN,131774,NaN,None,NaN,NaN,NaN,None
38472443,FR,FR,379974/15/185782,NaN,NaN,NaN,PTIT BASILE,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",2022-12-29T00:25:51Z,417ca1d7-3878-437d-a6a9-13d82f4ced78,https://www.auchan.fr/bebe/puericulture/chambr...,https://www.auchan.fr/ptit-basile-gigoteuse-et...,None,CA1129202,None,https://media.auchan.fr/4bbea478-5a65-44d3-b75...,1,0,0,0,0,0,NaN,NaN,NaN,Gigoteuse été jersey bio imprimé mini stars,NaN,"{""is_heard_on_radio"": false}",1 coloris,WEB,54,NaN,29.90,None,NaN,None,1,Auchan Drive,NaN,131774,NaN,None,NaN,NaN,NaN,None
38472444,FR,FR,379974/15/185782,NaN,NaN,NaN,PTIT BASILE,"['Tout pour bébé', 'Puériculture', 'Chambre bé...",2022-12-29T00:25:55Z,417ca1d7-3878-437d-a6a9-13d82f4ced78,https://www.auchan.fr/bebe/puericulture/chambr...,https://www.auchan.fr/ptit-basile-lot-x3-draps...,None,CA1306628,None,https://media.auchan.fr/25a25b98-be4e-4974-98c...,1,0,0,0,0,0,NaN,NaN,NaN,Lot x3 draps housse jersey lit bébé 100% coton...,NaN,"{""is_heard_on_radio"": false}",4 coloris,WEB,55,NaN,29.60,None,NaN,None,1,Auchan Drive,NaN,131774,NaN,None,NaN,NaN,NaN,None


In [12]:
# Spark session & context
spark = SparkSession.builder.master("spark://spark:7077") \
        .appName("jupyter-notebook-index2mongo") \
        .config("spark.driver.memory", "512m") \
        .config("spark.mongodb.input.uri", "mongodb://mongodb:27017/test.myCollection") \
        .config("spark.mongodb.output.uri", "mongodb://mongodb:27017/test.myCollection") \
        .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.2') \
        .getOrCreate()

spark

In [21]:

#pdf = pd.read_parquet('/home/jovyan/work/data/17_12_2022/')

files = glob.glob('/home/jovyan/work/data/**/*.parquet.zst')

In [ ]:
pdf = pd.concat([pd.read_parquet(fp) for fp in files])
len(pdf)

In [23]:


df = vaex.from_pandas(pdf, copy_index=False) # assuming you don't care about the index, which i think spark also does not have by default
df

[01/18/23 21:33:37] ERROR    error opening                                                          ]8;id=431264;file:///opt/conda/lib/python3.8/site-packages/vaex/__init__.py\__init__.py]8;;\:]8;id=360951;file:///opt/conda/lib/python3.8/site-packages/vaex/__init__.py#271\271]8;;\
                             '/home/jovyan/work/data/27_12_2022/2022-12-27.1.13392.parquet.zst'                    
                             Traceback (most recent call last):                                                    
                               File "/opt/conda/lib/python3.8/site-packages/vaex/__init__.py", line                
                             244, in open                                                                          
                                 ds = vaex.dataset.open(path, fs_options=fs_options, fs=fs,                        
                             **kwargs)                                                                             
                               File "/opt/conda/lib/python3.8/site-packages/vaex/dataset.py", line                 
                             95, in open                                                                           
                                 raise IOError(f'Cannot open {path} nobody knows how to read it.')                 
                             OSError: Cannot open                                                                  
                             /home/jovyan/work/data/27_12_2022/2022-12-27.1.13392.parquet.zst                      
                             nobody knows how to read it.                                                          

OSError: Cannot open /home/jovyan/work/data/27_12_2022/2022-12-27.1.13392.parquet.zst nobody knows how to read it.

# Write data in MongoDB

In [13]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")  

In [14]:
df = spark.read.option("overwriteSchema", "true").parquet('/home/jovyan/work/data/**/*.parquet.zst')
#df = spark.read.option("overwriteSchema", "true").parquet('/home/jovyan/work/data/21_12_2022/2022-12-21.1.131762.parquet.zst')

df

DataFrame[language: string, country: string, job_id: string, productCategoryAlt: double, productAltEAN: double, productPriceBase: float, productBrand: string, productCategory: string, productDate: string, crawlID: string, currentURL: string, productLinkDetail: string, productEAN: string, productID: string, productAltId: string, productImage: string, productIsAvailable: bigint, productIsBio: bigint, productIsFresh: bigint, productIsFreezer: bigint, productIsNew: bigint, promotion: bigint, isSponsored: double, productLotPriceBase: float, productLotPrice: float, productName: string, productNutriscore: double, other: string, productPackaging: string, typeCrawler: string, productPosition: bigint, productPricePrevious: float, productPrice: float, productPromotionText: string, productQuantityRating: bigint, productRating: string, enseigneID: string, enseigneName: string, productService: double, shopID: string, productStock: bigint, productUnit: string, productValueUnit: float, variant_master:

In [17]:
df.printSchema()

root
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- job_id: string (nullable = true)
 |-- productCategoryAlt: double (nullable = true)
 |-- productAltEAN: double (nullable = true)
 |-- productPriceBase: float (nullable = true)
 |-- productBrand: string (nullable = true)
 |-- productCategory: string (nullable = true)
 |-- productDate: string (nullable = true)
 |-- crawlID: string (nullable = true)
 |-- currentURL: string (nullable = true)
 |-- productLinkDetail: string (nullable = true)
 |-- productEAN: string (nullable = true)
 |-- productID: string (nullable = true)
 |-- productAltId: string (nullable = true)
 |-- productImage: string (nullable = true)
 |-- productIsAvailable: long (nullable = true)
 |-- productIsBio: long (nullable = true)
 |-- productIsFresh: long (nullable = true)
 |-- productIsFreezer: long (nullable = true)
 |-- productIsNew: long (nullable = true)
 |-- promotion: long (nullable = true)
 |-- isSponsored: double (nullable = tru

In [18]:
df.select('productName').collect()

Py4JJavaError: An error occurred while calling o54.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 37 in stage 2.0 failed 4 times, most recent failure: Lost task 37.3 in stage 2.0 (TID 1544) (172.20.0.2 executor 0): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2607)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGSch

In [5]:
d2 = df.withColumn("zip_code",df["zip_code"].cast(StringType()))

In [6]:
d2.printSchema()

root
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- job_id: string (nullable = true)
 |-- productCategoryAlt: double (nullable = true)
 |-- productAltEAN: double (nullable = true)
 |-- productPriceBase: float (nullable = true)
 |-- productBrand: string (nullable = true)
 |-- productCategory: string (nullable = true)
 |-- productDate: string (nullable = true)
 |-- crawlID: string (nullable = true)
 |-- currentURL: string (nullable = true)
 |-- productLinkDetail: string (nullable = true)
 |-- productEAN: string (nullable = true)
 |-- productID: string (nullable = true)
 |-- productAltId: string (nullable = true)
 |-- productImage: string (nullable = true)
 |-- productIsAvailable: long (nullable = true)
 |-- productIsBio: long (nullable = true)
 |-- productIsFresh: long (nullable = true)
 |-- productIsFreezer: long (nullable = true)
 |-- productIsNew: long (nullable = true)
 |-- promotion: long (nullable = true)
 |-- isSponsored: double (nullable = tru

In [7]:
d2.write.format("mongo").mode("overwrite").option("overwriteSchema", "true").save()

In [8]:
d2.printSchema()

root
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- job_id: string (nullable = true)
 |-- productCategoryAlt: double (nullable = true)
 |-- productAltEAN: double (nullable = true)
 |-- productPriceBase: float (nullable = true)
 |-- productBrand: string (nullable = true)
 |-- productCategory: string (nullable = true)
 |-- productDate: string (nullable = true)
 |-- crawlID: string (nullable = true)
 |-- currentURL: string (nullable = true)
 |-- productLinkDetail: string (nullable = true)
 |-- productEAN: string (nullable = true)
 |-- productID: string (nullable = true)
 |-- productAltId: string (nullable = true)
 |-- productImage: string (nullable = true)
 |-- productIsAvailable: long (nullable = true)
 |-- productIsBio: long (nullable = true)
 |-- productIsFresh: long (nullable = true)
 |-- productIsFreezer: long (nullable = true)
 |-- productIsNew: long (nullable = true)
 |-- promotion: long (nullable = true)
 |-- isSponsored: double (nullable = tru

# Read data in MongoDB

In [9]:
df = spark.read.format("mongo").load()
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- country: string (nullable = true)
 |-- crawlID: string (nullable = true)
 |-- currentURL: string (nullable = true)
 |-- enseigneID: string (nullable = true)
 |-- enseigneName: string (nullable = true)
 |-- job_id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- other: string (nullable = true)
 |-- productBrand: string (nullable = true)
 |-- productCategory: string (nullable = true)
 |-- productDate: string (nullable = true)
 |-- productEAN: string (nullable = true)
 |-- productID: string (nullable = true)
 |-- productImage: string (nullable = true)
 |-- productIsAvailable: long (nullable = true)
 |-- productIsBio: long (nullable = true)
 |-- productIsFreezer: long (nullable = true)
 |-- productIsFresh: long (nullable = true)
 |-- productIsNew: long (nullable = true)
 |-- productLinkDetail: string (nullable = true)
 |-- productName: string (nullable = true)
 |-- productPackaging: s

In [10]:
df.select("productName").show(5)

+--------------------+
|         productName|
+--------------------+
|Nature de France ...|
|Auchan Saucisses ...|
|Les Brasérades fa...|
|Masques à usage u...|
|      Pavé de saumon|
+--------------------+
only showing top 5 rows



In [1]:
import vaex
vdf = vaex.open('/home/jovyan/work/data/**/*.parquet.zst')
vdf

[01/18/23 19:57:35] ERROR    error opening                                                          ]8;id=502265;file:///opt/conda/lib/python3.8/site-packages/vaex/__init__.py\__init__.py]8;;\:]8;id=221070;file:///opt/conda/lib/python3.8/site-packages/vaex/__init__.py#271\271]8;;\
                             '/home/jovyan/work/data/17_12_2022/2022-12-17.1.115914.parquet.zst'                   
                             Traceback (most recent call last):                                                    
                               File "/opt/conda/lib/python3.8/site-packages/vaex/__init__.py", line                
                             244, in open                                                                          
                                 ds = vaex.dataset.open(path, fs_options=fs_options, fs=fs,                        
                             **kwargs)                                                                             
                               File "/opt/conda/lib/python3.8/site-packages/vaex/dataset.py", line                 
                             95, in open                                                                           
                                 raise IOError(f'Cannot open {path} nobody knows how to read it.')                 
                             OSError: Cannot open                                                                  
                             /home/jovyan/work/data/17_12_2022/2022-12-17.1.115914.parquet.zst                     
                             nobody knows how to read it.                                                          

                    ERROR    error opening '/home/jovyan/work/data/**/*.parquet.zst'                ]8;id=756306;file:///opt/conda/lib/python3.8/site-packages/vaex/__init__.py\__init__.py]8;;\:]8;id=142006;file:///opt/conda/lib/python3.8/site-packages/vaex/__init__.py#271\271]8;;\
                             Traceback (most recent call last):                                                    
                               File "/opt/conda/lib/python3.8/site-packages/vaex/__init__.py", line                
                             259, in open                                                                          
                                 dfs.append(vaex.open(filename, fs_options=fs_options, fs=fs,                      
                             convert=bool(convert), shuffle=shuffle, **kwargs))                                    
                               File "/opt/conda/lib/python3.8/site-packages/vaex/__init__.py", line                
                             244, in open                                                                          
                                 ds = vaex.dataset.open(path, fs_options=fs_options, fs=fs,                        
                             **kwargs)                                                                             
                               File "/opt/conda/lib/python3.8/site-packages/vaex/dataset.py", line                 
                             95, in open                                                                           
                                 raise IOError(f'Cannot open {path} nobody knows how to read it.')                 
                             OSError: Cannot open                                                                  
                             /home/jovyan/work/data/17_12_2022/2022-12-17.1.115914.parquet.zst                     
                             nobody knows how to read it.                                                          

OSError: Cannot open /home/jovyan/work/data/17_12_2022/2022-12-17.1.115914.parquet.zst nobody knows how to read it.